In [1]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint,CSVLogger
from tensorboard.plugins.hparams import api as hp

In [2]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [3]:
class SobolevNetwork(Model):
    def __init__(self, input_dim, num_hidden,init = None):
        super(SobolevNetwork, self).__init__()
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random.normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp1 = tf.keras.layers.Dropout(0.2)
        self.W2 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp2 = tf.keras.layers.Dropout(0.2)
        self.W3 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp3 = tf.keras.layers.Dropout(0.2)
        self.W4 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp4 = tf.keras.layers.Dropout(0.2)
        self.W5 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp5 = tf.keras.layers.Dropout(0.2)
        self.W6 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))
        self.dp6 = tf.keras.layers.Dropout(0.2)        
        self.W7 = tf.Variable(tf.random.normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.w = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        
    def call(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.w[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        out = self.dp1(out)
        W,b = self.w[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        out = self.dp2(out)
        W,b = self.w[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        out = self.dp3(out)
        W,b = self.w[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        out = self.dp4(out)
        W,b = self.w[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        out = self.dp5(out)
        W,b = self.w[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.w[-1]
        out = tf.matmul(out, W) + b
        return out


In [4]:
class LossCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        print("\n For Epoch Number {} the model has loss[mean_absolute_error] of {}".format(epoch+1, logs["loss"]))

    def on_batch_end(self, batch, logs):
        print("\n For Batch Number {} the model has loss[mean_absolute_error] of {}".format(batch+1, logs))

In [5]:
csv_callback = CSVLogger( 'logs.csv',separator=',',append=True)

In [6]:
es_callback = EarlyStopping( monitor='val_loss',min_delta=0,patience=2, verbose=1, mode="auto", baseline=None, restore_best_weights=False)

In [7]:
def schedular(epoch, lr):
    if epoch <= 3:
        learning_rate = lr
    else:
        learning_rate = lr * tf.math.exp(-0.1)
        learning_rate = learning_rate.numpy()
    

    # with train_writer.as_default():
    #     tf.summary.scalar('LearningRateScheduler', data= learning_rate, step= epoch)
    
    return learning_rate

schedular_callback = LearningRateScheduler(schedular, verbose=1)

In [8]:
num_hidden_1 = 12

In [9]:
custom_snn= SobolevNetwork(input_dim=16,num_hidden=num_hidden_1)

In [10]:
custom_snn.compile(optimizer=tf.keras.optimizers.Adam(),  
            loss='mean_absolute_error')

In [11]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [12]:
custom_snn.fit(dep_train, indep_train, validation_data=(dep_val,indep_val),batch_size = 1000, epochs = 8, callbacks=[schedular_callback,LossCallback(), csv_callback,tensorboard_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/8

 For Batch Number 1 the model has loss[mean_absolute_error] of {'loss': 1.289264440536499}
 1/43 [..............................] - ETA: 4:07 - loss: 1.2893
 For Batch Number 2 the model has loss[mean_absolute_error] of {'loss': 1.275917887687683}

 For Batch Number 3 the model has loss[mean_absolute_error] of {'loss': 1.2621989250183105}

 For Batch Number 4 the model has loss[mean_absolute_error] of {'loss': 1.2480359077453613}
 4/43 [=>............................] - ETA: 0s - loss: 1.2480  
 For Batch Number 5 the model has loss[mean_absolute_error] of {'loss': 1.2344616651535034}

 For Batch Number 6 the model has loss[mean_absolute_error] of {'loss': 1.221309781074524}

 For Batch Number 7 the model has loss[mean_absolute_error] of {'loss': 1.2079960107803345}
 7/43 [===>..........................] - ETA: 0s - loss: 1.2080
 For Batch Number 8 the model has loss[mean_absolute_error] of {'los

In [13]:
custom_snn.evaluate(dep_test, indep_test)

342/342 [==============================] - 1s 4ms/step - loss: 0.0022


0.002162031829357147

In [15]:
custom_snn.predict(dep_test)

342/342 [==============================] - 2s 5ms/step


array([[0.0103789 ],
       [0.01052278],
       [0.01038009],
       ...,
       [0.01052535],
       [0.01054573],
       [0.01016152]], dtype=float32)

In [15]:
%load_ext tensorboard
%tensorboard --logdir logs/fit


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 17416), started 5 days, 1:03:26 ago. (Use '!kill 17416' to kill it.)

In [17]:
%reload_ext tensorboard

In [22]:
from tensorboard.plugins.hparams import api as hp

In [18]:
def model_builder(hp):
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
  # hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#   model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
#   model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
#   model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model= SobolevNetwork(input_dim=16,num_hidden=hp_layer_1)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_absolute_error')
  
  return model

In [27]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='x')

INFO:tensorflow:Reloading Oracle from existing project dir\x\oracle.json


In [28]:
tuner.search(dep_train, indep_train, epochs=50, validation_split=0.2, callbacks=[es_callback])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
901               |?                 |layer_1
0.01              |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2
1051/1051 [==============================] - 90s 83ms/step - loss: 1.5470 - val_loss: 0.0431
Epoch 2/2
1051/1051 [==============================] - 93s 89ms/step - loss: 0.0095 - val_loss: 0.0036


KeyError: 'mse'

In [29]:
from tensorflow.keras import datasets

(X_train_reg, Y_train_reg), (X_test_reg, Y_test_reg) = datasets.boston_housing.load_data()

X_train_reg.shape, X_test_reg.shape, Y_train_reg.shape, Y_test_reg.shape

57026/57026 [==============================] - 0s 1us/step


((404, 13), (102, 13), (404,), (102,))

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

def build_model(hyperparams):
    model = Sequential()
    model.add(layers.Input(shape=(X_train_reg.shape[1],)))
    model.add(layers.Dense(units=hyperparams.Int("units_l1", 16, 50, step=16),
                           use_bias=hyperparams.Boolean("bias_l1"),
                           activation=hyperparams.Choice("act_l1", ["relu", "tanh"])
                          ))
    model.add(layers.Dense(units=hyperparams.Int("units_l2", 16, 50, step=16),
                           use_bias=hyperparams.Boolean("bias_l2"),
                           activation=hyperparams.Choice("act_l2", ["relu", "tanh"])
                          ))
    model.add(layers.Dense(1))

    optim=hyperparams.Choice("optimizer",["sgd","rmsprop","adam"])
    model.compile(optim, loss="mean_squared_error", metrics=["mean_squared_error"])

    return model

In [31]:
from keras_tuner import RandomSearch
from keras_tuner import Objective

tuner1 =  RandomSearch(hypermodel=build_model,
                      objective="val_mean_squared_error",
                      #objective=Objective(name="val_mean_squared_error",direction="min"),
                      max_trials=5,
                      #seed=123,
                      project_name="Regression",
                      overwrite=True
                    )

tuner1.search(X_train_reg, Y_train_reg, batch_size=32, epochs=10, validation_data=(X_test_reg, Y_test_reg))

Trial 5 Complete [00h 00m 07s]
val_mean_squared_error: 310.8663635253906

Best val_mean_squared_error So Far: 70.79254913330078
Total elapsed time: 00h 00m 36s
INFO:tensorflow:Oracle triggered exit


In [32]:
best_params = tuner1.get_best_hyperparameters()

best_params[0].values

{'units_l1': 32,
 'bias_l1': True,
 'act_l1': 'tanh',
 'units_l2': 32,
 'bias_l2': True,
 'act_l2': 'tanh',
 'optimizer': 'sgd'}

In [33]:
best_model = tuner1.get_best_models()[0]

best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                448       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________


In [40]:
HYPERBAND_MAX_EPOCHS = 150
EXECUTION_PER_TRIAL = 2
from keras_tuner.tuners import RandomSearch, Hyperband
tuner= Hyperband(build_model,
                   objective= 'val_mean_squared_error',
                   max_epochs=HYPERBAND_MAX_EPOCHS, #Set 100+ for good results
                   executions_per_trial=EXECUTION_PER_TRIAL,
                   directory= 'hyperband',
                   project_name='houseprices',
                   overwrite=True)

In [41]:

tuner.search(x= X_train_reg,
             y= Y_train_reg,
             epochs=100,
             batch_size= 64,
             validation_data= (X_test_reg, Y_test_reg),
             verbose=0,
             
            )
# tuner1.search(X_train_reg, Y_train_reg, batch_size=32, epochs=10, validation_data=(X_test_reg, Y_test_reg))

c:\Users\kiran.franklin\Anaconda3\lib\site-packages\keras_tuner\engine\metrics_tracking.py:105: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


INFO:tensorflow:Oracle triggered exit


In [50]:
tuner.get_best_hyperparameters()

In [44]:
# Retreive the optimal hyperparameters
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

In [49]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is {best_hps.get('units_l1')},
second layer is {best_hps.get('units_l2')} 
  


""")

# Evaluate the best model.
print(best_model.metrics_names)
loss, mae, mse = best_model.evaluate(X_test_reg, Y_test_reg)
print(f'loss:{loss} mae: {mae} mse: {mse}')


The hyperparameter search is complete. The optimal number of units in the 
first densely-connected layer is 32,
second layer is 32 
  



[]


ValueError: Creating variables on a non-first call to a function decorated with tf.function.